# Random Forest Regressor

---

**Источники:**

[sklearn.ensemble.RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

[1.11.2. Forests of randomized trees](https://scikit-learn.org/stable/modules/ensemble.html#forest)


[Random Forest Regression](https://medium.com/swlh/random-forest-and-its-implementation-71824ced454f)

[Painless Random Forest Regression in Python – Step-by-Step with Sklearn](https://onestopdataanalysis.com/python-random-forest-regression/)

[Random forest](https://ru.wikipedia.org/wiki/Random_forest)

[Бэггинг](https://ru.wikipedia.org/wiki/%D0%91%D1%8D%D0%B3%D0%B3%D0%B8%D0%BD%D0%B3)

[How to Visualize a Decision Tree from a Random Forest in Python using Scikit-Learn](https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c)

---


## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [2]:
# !conda install matplotlib numpy scikit-learn seaborn -y

In [3]:
import numpy as np

np.__version__

'1.19.2'

In [4]:
import pandas as pd

pd.__version__

'1.2.3'

In [5]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.__version__

'3.3.4'

In [6]:
import seaborn as sns

sns.__version__

'0.11.1'

## Загрузка данных

[Источник (FuelConsumption)](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

In [7]:
df = pd.read_csv("./../../data/FuelConsumptionCo2.csv")
df

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1063,2014,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264
1064,2014,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1065,2014,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260


## Разделение данных

In [8]:
from sklearn.model_selection import train_test_split

y = df['CO2EMISSIONS'].copy()
X = df[['ENGINESIZE']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Random Forest (Случайный лес) Regressor

[sklearn.ensemble.RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

Основная идея заключается в использовании большого ансамбля решающих деревьев (decision tree), каждое из которых само по себе даёт очень невысокое качество, но за счёт их большого количества результат получается хорошим.

Алгоритм применяется для задач **классификации, регрессии и кластеризации**.

**Достоинства:**
- Способность эффективно обрабатывать данные с большим числом признаков и классов.
- Нечувствительность к масштабированию (и вообще к любым монотонным преобразованиям) значений признаков.
- Одинаково хорошо обрабатываются как непрерывные, так и дискретные признаки. Существуют методы построения деревьев по данным с пропущенными значениями признаков.
- Существуют методы оценивания значимости отдельных признаков в модели.
- Внутренняя оценка способности модели к обобщению (тест по неотобранным образцам out-of-bag).
- Высокая параллелизуемость и масштабируемость.

**Недостатки:**
- Большой размер получающихся моделей. Требуется $O(K)$ памяти для хранения модели, где $K$ — число деревьев.

<center><img src="images/random_forest.png"/></center>

---

<center><img src="images/random_forest_2.png"/></center>

<center><img src="images/random_forest_regression_line.png"/></center>

## Обучение модели (Train)

In [9]:
# разделить независимую и зависимую переменные / train и test

from sklearn.model_selection import train_test_split

y = df['CO2EMISSIONS'].copy()
X = df.drop(['CO2EMISSIONS', 'MAKE', 'MODEL', 'VEHICLECLASS', 'TRANSMISSION', 'FUELTYPE'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Параметры RandomForestRegressor

- `n_estimators: int, default=100`
    - Количество деревьев в лесу.
    
    
- `criterion: {"mse", "mae"}, default="mse"`
    - Функция измерения качества разделения.
    - Поддерживаемые критерии: `"mse"` для среднеквадратичной ошибки, которая равна уменьшению дисперсии в качестве критерия выбора признаков, и `"mae"` для средней абсолютной ошибки.
    
    
- `max_depth: int, default=None`
    - Максимальная глубина дерева.
    - Если `None`, то узлы расширяются до тех пор, пока все листья не станут чистыми или пока все листья не будут содержать менее `min_samples_split` выборок.
    
   
- `min_samples_split: int или float, default=2`
    - Минимальное количество выборок, необходимое для разделения внутреннего узла:
        - Если `int`, то считает `min_samples_split` минимальным числом.
        - Если `float`, то `min_samples_split` - это дробь, а `ceil(min_samples_split * n_samples)` - минимальное количество выборок для каждого разделения.
        
        
- `min_samples_leaf: int or float, default=1`
    - Минимальное количество выборок, которое требуется для конечного узла.
    - Точка разделения на любой глубине будет учитываться только в том случае, если она оставляет не менее `min_samples_leaf` обучающих выборок в каждой из левой и правой ветвей. Это может иметь эффект сглаживания модели, особенно при регрессии.
    - Если `int`, то считает `min_samples_leaf` минимальным числом.
    - Если `float`, то `min_samples_leaf` - дробная часть, а `ceil(min_samples_leaf * n_samples)` - минимальное количество выборок для каждого узла.
    
    
- `min_weight_fraction_leaf: float, default=0.0`


- `max_features: {"auto", "sqrt", "log2"}, int or float, default="auto"`


- `max_leaf_nodes: int, default=None`
    - Выращиевает деревья с `max_leaf_nodes` способом "лучший первый" (best-first).
    - Лучшие узлы определяются как относительное уменьшение "загрязнения" (impurity).
    - Если `None`, то неограниченное количество конечных узлов.
    

- `min_impurity_decrease: float, default=0.0`


- `min_impurity_split: float, default=None`


- `bootstrap: bool, default=True`


- `oob_score: bool, default=False`


- `n_jobs: int, default=None`


- `random_state: int, RandomState instance or None, default=None`
    - Управляет как рандомизацией bootstrapping выборок, используемых при построении деревьев (если bootstrap = True), так и выборкой признаков, которые следует учитывать при поиске наилучшего разделения на каждом узле (если max_features < n_features).
    
  
- `verbose: int, default=0`


- `warm_start: bool, default=False`


- `ccp_alpha: non-negative float, default=0.0`


- `max_samples: int or float, default=None`

In [10]:
# импортировать пакет RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

# создать объект RandomForestRegressor
model = RandomForestRegressor(n_estimators=10, 
                              max_depth=5)

# обучить модель
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, n_estimators=10)

## Оценка качества модели (Evaluation)

In [11]:
# использовать обученную модель для предсказания на test выборке
y_predicted = model.predict(X_test)

In [12]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print("Mean absolute error: %.2f" % np.mean(np.absolute(y_predicted - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_predicted - y_test) ** 2))
print("R2-score: %.2f" % r2_score(y_true=y_test, y_pred=y_predicted))

Mean absolute error: 7.67
Residual sum of squares (MSE): 232.88
R2-score: 0.94


Для визуализации необходимо установить [`Graphviz`](https://graphviz.gitlab.io/download/).

In [13]:
from sklearn.tree import export_graphviz
import os

for i, est in enumerate(model.estimators_):
    export_graphviz(est, out_file=f'tmp/tree_{i}.dot', 
                    feature_names = X_train.columns.to_list(),
                    class_names = y_train.name,
                    rounded = True, proportion = False, 
                    precision = 2, filled = True)
    os.system(f'dot -Tpng tmp/tree_{i}.dot -o tmp/tree_{i}.png')